Erniyaz Ashuov Test Part

In [46]:
import pandas as pd
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
import seaborn as sns

In [47]:
# Excel file
file_path = 'Данные для тестового задания.xlsx'
data = pd.read_excel(file_path, sheet_name='Данные об аудитории')
ab_test_data = pd.read_excel(file_path, sheet_name='Данные АБ тестов')
listers_data = pd.read_excel(file_path, sheet_name='Листеры')

In [48]:
data['date'] = pd.to_datetime(data['date'], errors='coerce')



---



In [57]:
# 1. MAU (Monthly Active Users)
november_data = data[(data['date'] >= '2023-11-01') & (data['date'] <= '2023-11-30')]
MAU = november_data['user_id'].nunique()
print("MAU:", MAU)

MAU: 7639




---



In [58]:
#2. DAU (Daily Active Users)
DAU = november_data.groupby(november_data['date'].dt.date)['user_id'].nunique()

# Calculate the average DAU for November
average_dau = DAU.mean()
print(f"DAU: {average_dau}")

DAU: 560.4666666666667




---



In [59]:
# 3. Day 1 Retention
november_1_users = november_data[november_data['date'] == '2023-11-01']['user_id'].unique()

if len(november_1_users) > 0:
    november_2_users = november_data[(november_data['date'] == '2023-11-02') &
                                     (november_data['user_id'].isin(november_1_users))]['user_id'].nunique()
    day_1_retention = (november_2_users / len(november_1_users)) * 100
else:
    day_1_retention = "Not applicable (no users on November 1)"

print("Day 1 retention rate:", day_1_retention)

Day 1 retention rate: 26.64526484751204




---



In [60]:
# 5. Conversion rate
total_users_november = november_data['user_id'].nunique()

users_with_views = november_data[november_data['view_adverts'] > 0]['user_id'].nunique()

conversion_rate = (users_with_views / total_users_november) * 100
print(f"User conversion rate: {conversion_rate:.2f}%")

User conversion rate: 46.31%




---



In [61]:
# 6. Average number of advertisements viewed per user
average_views_per_user = november_data['view_adverts'].sum() / total_users_november
print(f"Average number of advertisements viewed per user: {average_views_per_user:.2f}")

Average number of advertisements viewed per user: 2.87




---



In [62]:
# 7. NPS (Net Promoter Score)
total_users = 2000
detractors = 500
promoters = 1200
passives = 300

percent_promoters = (promoters / total_users) * 100
percent_detractors = (detractors / total_users) * 100

NPS = percent_promoters - percent_detractors
print(f"NPS: {NPS:.2f}%")

NPS: 35.00%




---



In [66]:
# 8. A/B tests for ARPU (Average Revenue Per User)

arpu = ab_test_data.groupby(['experiment_num', 'experiment_group'])['revenue'].mean()
print("ARPU for each experiment and group:")
print(arpu)

for experiment in ab_test_data['experiment_num'].unique():
    control_group = ab_test_data[(ab_test_data['experiment_num'] == experiment) &
                                 (ab_test_data['experiment_group'] == 'control')]['revenue']
    test_group = ab_test_data[(ab_test_data['experiment_num'] == experiment) &
                              (ab_test_data['experiment_group'] == 'test')]['revenue']

    t_stat, p_value = ttest_ind(control_group, test_group, equal_var=False)

    print(f"\nExperiment {experiment}:")
    print(f"Control Group ARPU: {control_group.mean():.2f}")
    print(f"Test Group ARPU: {test_group.mean():.2f}")
    print(f"p-value: {p_value:.4f}")

    if p_value < 0.05:
        print(f"Result: Significant difference in ARPU for experiment {experiment}.")
        print("Recommendation: Consider implementing the changes in the test group.")
    else:
        print(f"Result: No significant difference in ARPU for experiment {experiment}.")
        print("Recommendation: Keep the current version as the test changes did not show significant improvement.")

ARPU for each experiment and group:
experiment_num  experiment_group
1               control             722.460215
                test                665.739583
2               control             704.653763
                test                332.929167
3               control             663.206452
                test                998.668750
Name: revenue, dtype: float64

Experiment 1:
Control Group ARPU: 722.46
Test Group ARPU: 665.74
p-value: 0.6890
Result: No significant difference in ARPU for experiment 1.
Recommendation: Keep the current version as the test changes did not show significant improvement.

Experiment 2:
Control Group ARPU: 704.65
Test Group ARPU: 332.93
p-value: 0.0011
Result: Significant difference in ARPU for experiment 2.
Recommendation: Consider implementing the changes in the test group.

Experiment 3:
Control Group ARPU: 663.21
Test Group ARPU: 998.67
p-value: 0.0603
Result: No significant difference in ARPU for experiment 3.
Recommendation: Keep the cur



---



In [64]:
# 9. Average revenue per user
average_revenue_per_user = listers_data['revenue'].mean()
print(f"Average revenue per user: {average_revenue_per_user:.2f}")

Average revenue per user: 30.70




---



In [65]:
# 10. Median age of users
median_age = listers_data['age'].median()
print(f"Median age of users: {median_age}")

Median age of users: 28.0
